# Parte 1: Entrenamiento, validación y selección


In [45]:
# Importamos las librerias que utilizaremos a lo largo del proyecto
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd

# Train-val-test-split

El primer es dar un vistazo a los datos con los que se cuenta. Iniciamos validando el tipo de dato de cada uno de ellos.

In [31]:
# Iniciamos leyendo el archivo de entrada
dataset = pd.read_csv("data_titanic_proyecto.csv")

# Contamos el número de regisitros y el numero de columnas que tiene el dataset
print(dataset.shape)

(891, 12)


In [32]:
# Vemos el tipo de dato que tiene cada columna en el dataset
dataset.dtypes

PassengerId             int64
Name                   object
Age                   float64
SibSp                   int64
Parch                   int64
Ticket                 object
Fare                  float64
Cabin                  object
Embarked               object
passenger_class        object
passenger_sex          object
passenger_survived     object
dtype: object

In [7]:
# Calculamos el portancaje de NaN en nuestro dataset
pd.DataFrame({'Porcentaje NaN': dataset.isnull().sum() * 100 / len(dataset)})

,Porcentaje NaN
PassengerId,0.000000
Name,0.000000
Age,19.865320
SibSp,0.000000
Parch,0.000000
Ticket,0.000000
Fare,0.000000
Cabin,77.104377
Embarked,0.224467
passenger_class,0.000000


Con base al análisis anterior, procedemos a descartar los features de: PassengerId, Name, Ticket y Cabin.

La columna 'Cabin' se descarta debido a que posee un 77% de datos NaN los cuales representan una gran incertidumbre. A continuación procedemos a definir 3 variables, una que contendrá las features que no se utilizarán, otra que contendrá las features categoricas y la última el nombre del feature a predecir, en este caso será passenger_survived.

In [28]:
def feature_engineering(ds, useless_features, categorical_features):
    
    # Eliminamos las features que no se utilizaran del dataset
    ds = ds.drop(useless_features, axis=1)
    
    # Normalizamos las variables que no son categoricas
    # Sustituiremos los valores NaN por el valor de la media
    imputer = Imputer(strategy="median")
    ds_numeric_features = ds.drop(categorical_features, axis=1)
    imputer.fit(ds_numeric_features)
    ds_imputer = imputer.transform(ds_numeric_features)
    ds_numeric_features = pd.DataFrame(ds_imputer, columns=ds_numeric_features.columns)
    
    # Normalizamos las variables categoricas
    ds_categorical_features = ds.drop(ds_numeric_features.columns, axis=1)
    ds_categorical_features_encoded = pd.DataFrame()
    
    # Sustituiremos los valores NaN por 'NaN'
    ds_categorical_features = ds_categorical_features.replace(np.nan, 'NaN', regex=True)
    
    # Usamos LabelEncoder para convertir nuestros valores categoricos a numericos
    ds_categorical_features = ds_categorical_features.apply(LabelEncoder().fit_transform)
    ds_categorical_features = pd.DataFrame(ds_categorical_features, columns= ds_categorical_features.columns)
    
    # Concatenamos nuestros dos DataFrames
    ds[categorical_features] = ds_categorical_features[categorical_features].values
    ds[ds_numeric_features.columns] = ds_numeric_features.values
    
    return ds

In [33]:
# Aplicamos feature engineering para normalizar los datos a una forma que no sea útil
useless_features = np.array(['PassengerId','Name','Ticket', 'Cabin'])
categorical_features = np.array(['passenger_sex','Embarked','passenger_class','passenger_survived'])
normalized_dataset = feature_engineering(dataset, useless_features, categorical_features)

print (normalized_dataset)

      Age  SibSp  Parch      Fare  Embarked  passenger_class  passenger_sex  \
0    22.0    1.0    0.0    7.2500         3                0              1   
1    38.0    1.0    0.0   71.2833         0                2              0   
2    26.0    0.0    0.0    7.9250         3                0              0   
3    35.0    1.0    0.0   53.1000         3                2              0   
4    35.0    0.0    0.0    8.0500         3                0              1   
5    28.0    0.0    0.0    8.4583         2                0              1   
6    54.0    0.0    0.0   51.8625         3                2              1   
7     2.0    3.0    1.0   21.0750         3                0              1   
8    27.0    0.0    2.0   11.1333         3                0              0   
9    14.0    1.0    0.0   30.0708         0                1              0   
10    4.0    1.0    1.0   16.7000         3                0              0   
11   58.0    0.0    0.0   26.5500         3         

C:\Users\Chofo\Anaconda3\envs\galileo_python\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


Luego que hemos hecho feature_engineering sobre nuestra data para normalizarla, procedemos a crear nuestros distintos datasets que utilizaremos para entrenar, validar y probar.

In [51]:
# Hacemos una copia de nuestro dataset normalizado sin incluir la variable a predecir
ds_x = normalized_dataset.drop('passenger_survived', axis=1)

predict_feature = 'passenger_survived'
ds_y = normalized_dataset[predict_feature].copy()

# Separamos nuestros datos en datos de prueba y datos de entrenamiento
# Utiliazaremos el 80% como datos de entrenamiento
x_train, x_test, y_train, y_test = train_test_split(ds_x, ds_y, train_size=0.8)

# Utilizamos los datos de entrenamiento para extraer una porción para datos de validación.
# Utilizaremos un 20% del dataset de entrenamiento para datos de validación
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, train_size=0.85)

print('Shape de los datos de entrenamiento: ', x_train.shape, y_train.shape)
print('Shape de los datos de validación', x_validation.shape, y_validation.shape)
print('Shape de los datos de prueba', x_test.shape, y_test.shape)

Shape de los datos de entrenamiento:  (605, 7) (605,)
Shape de los datos de validación (107, 7) (107,)
Shape de los datos de prueba (179, 7) (179,)
